In [2]:
import numpy as np

In [3]:
def calculate_potential(m):
    u, v = np.empty(m.shape[0]), np.empty(m.shape[1])
    u[:] = np.nan
    u[0] = 0
    v[:] = np.nan
    # calculate
    while np.isnan(u).any() or np.isnan(v).any():
        for i, u_i in enumerate(u):
            if not np.isnan(u_i):
                for j, c in enumerate(m[i, :]):
                    if c != 0 and np.isnan(v[j]):
                        v[j] = c - u_i
                        for k, row_element in enumerate(m[:, j]):
                            if k != i and row_element != 0 and np.isnan(u[k]):
                                u[k] = row_element - v[j]
    return u, v


In [4]:
def search_way(x, start, cur, direct='H', start_b=True):
    if direct=='W' and start[1] == cur[1] and start_b is not True:
        return [cur]
    if direct == 'H':
        row = np.where(x[cur[0], :] > 0)[0]
        if row.size == 0:
            return None
        for index_c in row:
            if index_c == cur[1]:
                continue
            return_v = search_way(x, start, [cur[0], index_c], 'W', False)
            if type(return_v) is list:
                return_v.append(cur)
                return return_v

        return None

    elif direct == 'W':
        col = np.where(x[:, cur[1]] > 0)[0]
        if col.size == 0:
            return None
        for index_r in col:
            if index_r == cur[0]:
                continue
            return_v = search_way(x, start, [index_r, cur[1]], 'H', False)
            if type(return_v) is list:
                return_v.append(cur)
                return return_v

        return None


In [27]:
def transport_task(user_in=False):
    if user_in:
        pass
    else:
        c = np.array([[29, 4, 7, 6, 16], [21, 13, 25, 21, 7], [20, 10, 12, 6, 2], [17, 7, 4, 6, 19]])
        a = np.array([14, 14, 14, 18])
        b = np.array([12, 12, 12, 12, 12])

        #c = np.array([[10, 17, 9, 20, 30], [13, 4, 24, 26, 26], [22, 24, 30, 27, 29], [8, 26, 7, 28, 9]])
        #a = np.array([15, 15, 19, 11])
        #b = np.array([9, 24, 9, 9, 9])
    x = np.zeros(c.shape)

    # опорный план метод северо запада
    j = 0
    for i in range(len(a)):
        while True:
            if a[i] > b[j]:
                x[i, j] = b[j]
                a[i] -= b[j]
                j += 1
            elif a[i] < b[j]:
                x[i, j] = a[i]
                b[j] -= a[i]
                break
            elif a[i] == b[j]:
                x[i, j] = a[i]
                j += 1
                break
    # потенциалы
    while True:
        u, v = calculate_potential(np.where(x, c, 0))
        temp_var = u[:, np.newaxis] + v - c
        m = np.max(temp_var[temp_var >= 0])
        if m == 0:
            break
        start = [i[0] for i in np.where(temp_var == m)]
        coords = search_way(x, start, start)
        coords = list(reversed(coords))
        temp_var = []
        for coord in coords:
            temp_var.append(x[coord[0], coord[1]])
        change = min(temp_var[1::2])
        temp_var[1::2] -= change
        temp_var[::2] += change
        for i in range(len(temp_var)):
            x[coords[i][0], coords[i][1]] = temp_var[i]
    return x, np.sum(x*c)


In [22]:
def user_input():
    try:
        a_c = int(input('количество пунктов отправления: '))
        check(a_c, 100, 'слишком большое пунктов отправления', 2, 
              'слишком мальнькое количество пунктов отправления')
        b_c = int(input('количество пунктов назначения: '))
        check(b_c, 100, 'слишком большое количество пунктов назначения', 2, 
              'слишком маленькое количество пунктов назначения')
        a, b = [], []
        c = []
        for i in range(a_c):
            a_i = int(input(f'единиц груза в пункте отправления №{i+1}: '))
            check(a_i, 100, f'слишком больое количество груза в пункте отправления  №{i+1}', 1, 
                  f'слишком маленькое количество груза в пункте отправлени №{i+1}')         
            a.append(a_i)
            c_i = []
            for j in range(b_c):
                c_i_j = int(input(f'стоимость перевозки из пункта №{i+1} в пункт №{j+1}: '))
                check(c_i_j, 100, f'слишком большая стоимость перевозки ', 1,
                      f'слишком маленькая стоимость перевозки')
                c_i.append(c_i_j)
            c.append(c_i)
        print('необходимое количество груза в ...')
        for i in range(b_c):
            b_i = int(input(f'... пунктке назначения №{i+1}: '))
            check(b_i, 100, f'слишком большое значение в пункте назначения №{i+1}', 1,
                      f'слишком маленькое значение в пункте назначения №{i+1}')
            b.append(b_i)
        if sum(a) != sum(b):
            raise Exception('не выполняеться необходимое и достаточное условие разрешимости задачи\n sum(a) != sum(b)')
        
        return np.array(c), np.array(a), np.array(b)
    
    except ValueError:
        print('error = тип не совпадает с нужным')
    except Exception as e:
        print('error =', e)
    finally:
        answer = int(input('повторить ввод? 1 - yes; 0 - no: \n'))
        if answer:
            print('\n------------------------------------------------------------\n')
            return user_input()

In [23]:
def check(v, max_v, max_err_m, min_v, min_err_m):
    
    if v > max_v:
        raise Exception(max_err_m)
    if v < min_v:
        raise Exception(min_err_m)


In [24]:
user_input()

количество пунктов отправления: 2
количество пунктов назначения: 2
единиц груза в пункте отправления №1: 3
стоимость перевозки из пункта №1 в пункт №1: 1
стоимость перевозки из пункта №1 в пункт №2: 1
единиц груза в пункте отправления №2: 3
стоимость перевозки из пункта №2 в пункт №1: 1
стоимость перевозки из пункта №2 в пункт №2: 1
необходимое количество груза в ...
... пунктке назначения №1: 2
... пунктке назначения №2: 2
error = не выполняеться необходимое и достаточное условие разрешимости задачи
 sum(a) != sum(b)
повторить ввод? 1 - yes; 0 - no: 
1

------------------------------------------------------------

количество пунктов отправления: -1
error = слишком мальнькое количество пунктов отправления
повторить ввод? 1 - yes; 0 - no: 
1

------------------------------------------------------------

количество пунктов отправления: a
error = тип не совпадает с нужным
повторить ввод? 1 - yes; 0 - no: 
1

------------------------------------------------------------

количество пунктов 

(array([[1, 1],
        [2, 2]]), array([2, 3]), array([4, 1]))

In [31]:
x, f = transport_task()
print(f'минимальные затраты F = {f},\nмассив пути доставки = \n{x}')

минимальные затраты F = 454.0,
массив пути доставки = 
[[ 0. 12.  0.  2.  0.]
 [12.  0.  0.  0.  2.]
 [ 0.  0.  0.  4. 10.]
 [ 0.  0. 12.  6.  0.]]
